In [5]:
#Task 4.1

import sqlite3

connection = sqlite3.connect("photogram.db")
# 1 mark for creating both tables
# 1 mark for correct fields for both tables (including correct Primary keys)
connection.execute('''CREATE TABLE 'User'( 
                    'UserID' INTEGER,
                    'Name' TEXT NOT NULL,
                    'Gender' TEXT NOT NULL,
                    'Country' TEXT NOT NULL,
                    PRIMARY KEY('UserID'))''')

#Total 2 marks: 1 mark each for each correct foreign keys
connection.execute("CREATE TABLE 'Following'(\
                    'UserID' INTEGER,\
                    'Following_UserID' INTEGER,\
                    FOREIGN KEY('Following_UserID') REFERENCES 'User'('UserID'),\
                    PRIMARY KEY('UserID','Following_UserID'),\
                    FOREIGN KEY('UserID') REFERENCES 'User'('UserID'))")

connection.commit()
connection.close()

#1 mark for successfully creating the database

In [6]:
#Task 4.2

import sqlite3
import csv

# create database connection
connection = sqlite3.connect("photogram.db")

with open("USERS.csv", "r", newline="") as infile: #1 mark for opening and closing file correctly for both USERS.csv and FOLLOWING.csv files
    records = csv.reader(infile, delimiter = ",")  #1 mark for csv reader or other methods of splitting the data for both USERS.csv and FOLLOWING.csv files
    for record in records: #1 mark for loop for extracting data for both USERS.csv and FOLLOWING.csv files     
        userid = int(record[0]) 
        name = record[1]
        gender = record[2]
        country = record[3]
            
        # execute SQL query
        connection.execute("INSERT INTO User (UserID, Name, Gender, Country) VALUES (?,?,?,?)", (userid, name, gender,country)) #1 mark for correct query and correct values to insert data into User table

        
with open("FOLLOWING.csv", "r", newline="") as infile:
    records = csv.reader(infile, delimiter = ",")
    for record in records:      
        userid = int(record[0])
        
        number_following = len(record) - 1
        for i in range(number_following): #1 mark for extracting people each user is following (must be dynamic as different users follow different number of people and different people)
            following_userid = int(record[1+i])
            
            # execute SQL query
            connection.execute("INSERT INTO Following (UserID, Following_UserID) VALUES (?,?)", (userid, following_userid)) #1 mark for correct query and correct values to insert data into Following table
       

        
connection.commit()
connection.close()
#1 mark for successfully inserting any query into database

In [2]:
#Task 4.4

import pymongo
import sqlite3

client = pymongo.MongoClient("127.0.0.1", 27017) #1 mark for connecting to PyMongo, creating db and collection
db = client.get_database("photogram")
db.drop_collection("users")
coll = db.get_collection("users")

#connecting to SQLite to get the data
# create database connection
connection = sqlite3.connect("photogram.db")

# execute SQL query
cursor = connection.execute("SELECT * FROM User") #1 mark for extracting the data from photogram.db or the relevant csv files

records = cursor.fetchall()

for record in records:
    userid = int(record[0])
    following_list = []
    cursor2 = connection.execute("SELECT Following_UserID FROM Following WHERE UserID = ?", (userid,))
    records2 = cursor2.fetchall()
    for record2 in records2:
        following_list.append(record2[0])
    coll.insert_one({"UserID":int(record[0]), "Name":record[1], "Gender":record[2], "Country":record[3], "Following":following_list})
    #Total: 2 marks, 2 marks for inserting the records correctly into PyMongo. If insert successfully but data is wrong, 1 mark 
    
result = coll.find()
print("All documents in users collection:")
for document in result:
    print(document)

connection.close()
client.close()

All documents in users collection:
{'_id': ObjectId('6477b3fd4c3090994c02ee74'), 'UserID': 1, 'Name': 'Low Sing Yu', 'Gender': 'Female', 'Country': 'Singapore', 'Following': [2, 3, 5, 7]}
{'_id': ObjectId('6477b3fd4c3090994c02ee75'), 'UserID': 2, 'Name': 'Goh Kai De', 'Gender': 'Male', 'Country': 'Singapore', 'Following': [1, 4, 5, 8, 10]}
{'_id': ObjectId('6477b3fd4c3090994c02ee76'), 'UserID': 3, 'Name': 'Mohammad Rizal', 'Gender': 'Male', 'Country': 'Singapore', 'Following': [1, 4, 9]}
{'_id': ObjectId('6477b3fd4c3090994c02ee77'), 'UserID': 4, 'Name': 'Indra Ravindran', 'Gender': 'Female', 'Country': 'Singapore', 'Following': []}
{'_id': ObjectId('6477b3fd4c3090994c02ee78'), 'UserID': 5, 'Name': 'Veenod Gunalan', 'Gender': 'Male', 'Country': 'Malaysia', 'Following': [3, 8, 9]}
{'_id': ObjectId('6477b3fd4c3090994c02ee79'), 'UserID': 6, 'Name': 'John Ajllin', 'Gender': 'Male', 'Country': 'Malaysia', 'Following': [3, 4, 7]}
{'_id': ObjectId('6477b3fd4c3090994c02ee7a'), 'UserID': 7, 'Nam